In [1]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Flatten, Convolution2D, MaxPooling2D,Dropout, Reshape, LSTM, BatchNormalization,TimeDistributed
from keras.layers import Conv2D
from keras.optimizers import Adam #, RMSprop, SGD
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from time import time

import pickle
import random
import numpy as np
import sys
import h5py
import glob
import os

Using TensorFlow backend.


In [2]:
batch_size =  16 #multiple of 1290/5 (recording length)
nb_classes = 18
rows, cols = 431, 128   
nb_epoch = 200
pool_size = (5,5)                  # size of pooling area for max pooling
prob_drop_conv = 0.3                # drop probability for dropout @ conv layer
prob_drop_hidden = 0.3              # drop probability for dropout @ fc layer

In [21]:
#might need to rewrite these



def fileLists():
    trainlist=[]
    validationlist=[]
    testlist=[]
    evalSetupFiles='..\\..\\CASAdatasets\\DCASE18_ASCT1\\TUT-urban-acoustic-scenes-2018-development\\evaluation_setup\\*.txt'
    txtfilelist=glob.glob(evalSetupFiles)
    for txt in txtfilelist:
        if '_location' not in txt:
            continue
        with open(txt,'r') as evaltxtfile:
            for line in evaltxtfile.readlines():
                line=line.strip().split('\t')[0]
                if 'train' in txt:
                    trainlist.append(line.replace('audio','logMelSpec').replace('.wav','_aggScenes.pckl').replace('/','\\'))
                elif 'test' in txt:
                    testlist.append(line.replace('audio','logMelSpec').replace('.wav','_aggScenes.pckl').replace('/','\\'))
                else:
                    validationlist.append(line.replace('audio','logMelSpec').replace('.wav','_aggScenes.pckl').replace('/','\\'))
    print('trainfiles: ', str(len(trainlist)))
    print('validationfiles: ', str(len(validationlist)))
    print('testfiles: ', str(len(testlist)))
    return trainlist,validationlist,testlist

labelRef={'inside-barcelona':0, 'outside-barcelona':1, 'transport-barcelona':2, 
          'inside-helsinki':3, 'outside-helsinki':4, 'transport-helsinki':5,
          'inside-london':6, 'outside-london':7,'transport-london':8,
          'inside-paris':9, 'outside-paris':10,'transport-paris':11,
          'inside-stockholm':12, 'outside-stockholm':13,'transport-stockholm':14,
          'inside-vienna':15, 'outside-vienna':16,'transport-vienna':17}

def getData(flist):
    pth='..\\..\\CASAdatasets\\DCASE18_ASCT1\\TUT-urban-acoustic-scenes-2018-development\\'
    X_=np.zeros(((len(flist)),128,431))
    Y_=np.zeros(((len(flist)),nb_classes))
    for i,tfile in enumerate(flist):
        with open(pth+tfile, "rb" ) as scenesample:
            fv=pickle.load(scenesample)
        X_[i,:,:]=fv
        scene=''.join(tfile.split('\\')[1]).split('-')[0]
        location=''.join(tfile.split('\\')[1]).split('-')[1]
        if scene in ['airport','shopping_mall','metro_station']:
            scene='inside-'+location
        elif scene in ['street_pedestrian','public_square','street_traffic','park']:
            scene='outside-'+location
        else:
            scene='transport-'+location
        Y_[i,labelRef[scene]]=1
    return X_, Y_
#trainlist,validationlist,testlist=fileLists()
#X_train, Y_train = getData(trainlist)
#X_val, Y_val = getData(validationlist)
#X_test, Y_test = getData(testlist)

print(' data sorted')


 data sorted


In [5]:
###### Convolutional model
def compileCRNN(cols,rows,nb_classes=1):
    model = Sequential()
    # conv1 layer
    model.add(Conv2D(32, (7, 7), padding='same', activation='relu', input_shape=(cols,rows,1)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=pool_size, strides=(2), padding='same'))
    model.add(Dropout(prob_drop_conv))

    # conv2 layer
    model.add(Conv2D(64, (7, 7), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(4,7), strides=(2), padding='same'))
    model.add(Dropout(prob_drop_conv))

    # conv3 layer
    model.add(Conv2D(128, (2, 2), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=pool_size, strides=(2), padding='same'))
    model.add(BatchNormalization())
    model.add(Dropout(prob_drop_conv))

    model.add(Flatten())
    # fc1 layer
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(prob_drop_hidden))
    model.add(BatchNormalization())
   
    # fc2 layer
    model.add(Dense(nb_classes, activation='softmax'))
    opt = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

model = compileCRNN(cols,rows,nb_classes=nb_classes)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 431, 32)      1600      
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 431, 32)      128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 216, 32)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 216, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 216, 64)       100416    
_________________________________________________________________
batch_normalization_2 (Batch (None, 64, 216, 64)       256       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 108, 64)       0         
__________

In [6]:
# Train
#fold=1
def train_network(model, model_name, X_train, Y_train, X_val, Y_val, nb_epoch, validationsplit_size, batchsize, early_stoping_patience, output_folder):
    checkpointer = ModelCheckpoint(filepath=os.path.join(output_folder,model_name + '.hdf5'),save_best_only=True)
    early_stopping = EarlyStopping(monitor='val_loss', patience=early_stoping_patience)
    tensorboard = TensorBoard(log_dir="logs\\{}".format(time()))
    Callbacks=[checkpointer,  tensorboard] #early_stopping,
  #  print(samples)
    steps=int(samples/batchsize)
    validationsteps=int(validationsplit_size/batchsize)
    history = model.fit(X_train, Y_train, epochs=nb_epoch, callbacks=Callbacks, batch_size=batch_size, validation_data=(X_val, Y_val), shuffle=True, verbose=1)
    return history,model

def buildModel(savemodelfilename, samples,model,X_train, Y_train,X_val, Y_val):
    valSplit_size = int(samples/4)
    early_stoping_patience=5
    history,model = train_network(model, 'models\\groupedScenePairs', X_train, Y_train, X_val, Y_val, nb_epoch, valSplit_size, batch_size, early_stoping_patience,'.')
    model.save_weights(savemodelfilename)
    return model, history


In [22]:
from datetime import datetime
batch_size =  16 #multiple of 1290/5 (recording length)
nb_classes = 18
rows, cols = 431, 128  


trainlist,validationlist,testlist=fileLists()
print('lists sorted')
X_tr, Y_train = getData(trainlist)
X_v, Y_val = getData(validationlist)
X_te, Y_test = getData(testlist)
print('data obtained')
X_train=np.expand_dims(X_tr,axis=3)
X_val=np.expand_dims(X_v,axis=3)
X_test=np.expand_dims(X_te,axis=3)
print(X_train.shape, X_val.shape, X_test.shape)
print('data sorted')
model = compileCRNN(cols,rows,nb_classes=nb_classes)
#model=load_model('models\\best_model60_simple.hdf5')
print('model compiled')
savemodelfilename='models\\groupedScenePairs.testsave'
samples=(X_train.shape[0])
print(datetime.now())
models,histories= buildModel(savemodelfilename, samples,model,X_train,Y_train,X_val, Y_val)
print(datetime.now())
print('model saved')

trainfiles:  6122
validationfiles:  2518
testfiles:  2518
lists sorted
data obtained
(6122, 128, 431, 1) (2518, 128, 431, 1) (2518, 128, 431, 1)
data sorted
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 128, 431, 32)      1600      
_________________________________________________________________
batch_normalization_5 (Batch (None, 128, 431, 32)      128       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 64, 216, 32)       0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 64, 216, 32)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 64, 216, 64)       100416    
_________________________________________________________________
batch_normalization_6 (Batch (None, 64, 216, 64)   

6122/6122 [==============================] - 24s 4ms/step - loss: 0.6135 - acc: 0.8071 - val_loss: 2.5160 - val_acc: 0.3300
Epoch 41/200
6122/6122 [==============================] - 24s 4ms/step - loss: 0.4646 - acc: 0.8639 - val_loss: 3.3937 - val_acc: 0.3435
Epoch 42/200
6122/6122 [==============================] - 24s 4ms/step - loss: 0.4269 - acc: 0.8747 - val_loss: 2.8708 - val_acc: 0.3169
Epoch 43/200
6122/6122 [==============================] - 24s 4ms/step - loss: 0.4398 - acc: 0.8672 - val_loss: 2.5670 - val_acc: 0.3852
Epoch 44/200
6122/6122 [==============================] - 24s 4ms/step - loss: 0.4082 - acc: 0.8821 - val_loss: 2.2696 - val_acc: 0.4110
Epoch 45/200
6122/6122 [==============================] - 24s 4ms/step - loss: 0.6698 - acc: 0.7885 - val_loss: 4.1054 - val_acc: 0.2216
Epoch 46/200
6122/6122 [==============================] - 24s 4ms/step - loss: 0.5410 - acc: 0.8298 - val_loss: 4.7450 - val_acc: 0.2649
Epoch 47/200
6122/6122 [==============================

Epoch 100/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.3236 - acc: 0.9013 - val_loss: 6.3934 - val_acc: 0.2172
Epoch 101/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.2237 - acc: 0.9304 - val_loss: 2.8056 - val_acc: 0.3745
Epoch 102/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.1744 - acc: 0.9492 - val_loss: 3.7669 - val_acc: 0.4031
Epoch 103/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.1798 - acc: 0.9456 - val_loss: 2.5348 - val_acc: 0.4631
Epoch 104/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.1522 - acc: 0.9564 - val_loss: 3.2398 - val_acc: 0.3817
Epoch 105/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.1423 - acc: 0.9587 - val_loss: 2.1285 - val_acc: 0.4519
Epoch 106/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.1422 - acc: 0.9556 - val_loss: 2.9438 - val_acc: 0.4559
Epoch 107/200
6122/6122 [=========

6122/6122 [==============================] - 23s 4ms/step - loss: 0.0981 - acc: 0.9680 - val_loss: 2.2438 - val_acc: 0.4857
Epoch 160/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.0938 - acc: 0.9713 - val_loss: 5.5561 - val_acc: 0.3713
Epoch 161/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.0948 - acc: 0.9709 - val_loss: 2.3773 - val_acc: 0.4464
Epoch 162/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.1920 - acc: 0.9378 - val_loss: 4.6710 - val_acc: 0.2077
Epoch 163/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.1921 - acc: 0.9371 - val_loss: 2.5155 - val_acc: 0.4595
Epoch 164/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.1152 - acc: 0.9654 - val_loss: 3.0455 - val_acc: 0.4527
Epoch 165/200
6122/6122 [==============================] - 23s 4ms/step - loss: 0.0985 - acc: 0.9703 - val_loss: 2.4032 - val_acc: 0.4567
Epoch 166/200
6122/6122 [=======================